In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/prop-1.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0      prop-1        4    2    4    0    2    5     1   0   2  ...  1.000000   
1      prop-1        4   12    4    1   19   44    66   2  18  ...  0.000000   
2      prop-1        4   10    4    0   13   48    45   4  13  ...  1.000000   
3      prop-1        4    2    3    0    5    7     1   0   5  ...  1.000000   
4      prop-1        4    8    1    0    4   38     0   3   1  ...  0.500000   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
18466  prop-1      185   17    1    0   24   67    96   9  15  ...  0.150000   
18467  prop-1      185    9    1    0   25   49    14   7  18  ...  0.333333   
18468  prop-1      185    6    3    0   10   29     0   0  10  ...  0.800000   
18469  prop-1      185    5    5    0   28    9    10  26   2  ...  1.000000   
18470  prop-1      185   11    1    0   54   34     9  51   3  ...  0.666667   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0        0  0.941176  0.750000   1    1  10.500000       3  1.5000    0  
1        0  0.905172  0.350000   2    3  20.583333       8  2.0833    0  
2        0  0.918919  0.500000   2    3  19.100000       4  1.2000    0  
3        0  0.933333  0.750000   1    1  11.500000       4  2.0000    1  
4        0  0.000000  0.285714   0    0  30.625000       2  0.8750    0  
...    ...       ...       ...  ..  ...        ...     ...     ...  ...  
18466    0  0.000000  0.250000   0    0  41.529412       6  2.6471    1  
18467    0  0.000000  0.319444   0    0  29.888889       3  1.3333    0  
18468    0  0.818182  0.400000   0    0  45.500000       6  2.1667    0  
18469    0  0.962963  0.650000   1    1   4.400000       1  0.2000    0  
18470    1  0.000000  0.400000   0    0  25.545455       6  1.9091    0  

[18471 rows x 23 columns]

In [4]:
ant_master = ant_master.loc[ant_master['version'] == 121]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...  dam  moa  \
10152  prop-1      121    0    1    0    0    0     0   0   0  ...  0.0    0   
10153  prop-1      121    0    1    0    0    0     0   0   0  ...  0.0    0   
10154  prop-1      121    3    1    0    5   14     1   0   5  ...  0.5    0   
10155  prop-1      121    9    1    0   18   22    36   6  12  ...  0.0    0   
10156  prop-1      121    0    1    0    0    0     0   0   0  ...  0.0    0   
...       ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...  ...  ...   
13145  prop-1      121    2    1    0    5   12     1   0   5  ...  0.0    0   
13146  prop-1      121    1    1    0    0    5     0   0   0  ...  0.0    0   
13147  prop-1      121    1    1    0    0    5     0   0   0  ...  0.0    0   
13148  prop-1      121    1    1    0    0    5     0   0   0  ...  0.0    0   
13149  prop-1      121    1    1    0    0    5     0   0   0  ...  0.0    0   

       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
10152  0.0  0.000000   0    0   0.000000       0  0.0000    0  
10153  0.0  0.000000   0    0   0.000000       0  0.0000    0  
10154  0.0  0.500000   0    0  17.666667       1  0.3333    0  
10155  0.0  0.305556   0    0  11.888889       4  1.4444    1  
10156  0.0  0.000000   0    0   0.000000       0  0.0000    0  
...    ...       ...  ..  ...        ...     ...     ...  ...  
13145  0.0  0.625000   0    0  26.000000       4  2.0000    0  
13146  0.0  0.000000   0    0  12.000000       0  0.0000    0  
13147  0.0  0.000000   0    0  12.000000       0  0.0000    0  
13148  0.0  0.000000   0    0  12.000000       0  0.0000    0  
13149  0.0  0.000000   0    0  12.000000       0  0.0000    0  

[2998 rows x 23 columns]

In [5]:
train, test = train_test_split(ant_master, test_size=0.2)

In [6]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [7]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3   loc       dam  \
11757    1    4    0    3    2     0   2   1    1  2.000000     5  0.000000   
12783    1    1    0    3    1     0   1   2    1  2.000000     1  0.000000   
11552    2    3    0    5    4     1   0   5    2  2.000000     8  0.000000   
12636    3    2    0    7   13     1   0   7    2  1.000000    59  0.333333   
12643   34    1    0   83  225   517  17  68   20  0.928030  2176  0.500000   
...    ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   ...       ...   
10745   14    4    0    7   32    79   1   6   10  0.961538   128  0.750000   
10621    0    1    0    0    0     0   0   0    0  2.000000     2  0.000000   
11786    9    2    0   11   49    16   1  10    3  0.892857   356  0.857143   
12312    0    1    0    4    0     0   4   0    0  2.000000    64  0.000000   
12879    6    3    0    8   22     9   0   8    6  0.866667   103  0.333333   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  
11757    0  1.000000  1.000000   0    0   4.000000       0  0.0000  
12783    0  0.000000  1.000000   0    0   0.000000       1  1.0000  
11552    0  0.857143  0.666667   1    1   3.000000       1  0.5000  
12636    0  0.666667  0.666667   1    1  17.666667       1  0.3333  
12643    0  0.000000  0.253444   0    0  62.764706      55  3.8235  
...    ...       ...       ...  ..  ...        ...     ...     ...  
10745    0  0.942029  0.234694   4   15   7.857143       2  1.2143  
10621    0  0.000000  0.000000   0    0   0.000000       0  0.0000  
11786    0  0.695652  0.375000   2    5  37.777778      14  2.7778  
12312    0  0.000000  0.000000   0    0   0.000000       0  0.0000  
12879    0  0.886364  0.333333   2    6  15.666667       6  2.0000  

[2398 rows x 20 columns]

In [8]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3   loc       dam  \
12617    7    3    0   29   54     7   0  29    6  0.869048   357  0.500000   
10965   18    1    2   51  144   103   5  48    7  0.931373  1216  0.333333   
12993    2    1    0    5   11     1   0   5    2  2.000000    46  0.000000   
12869    6    3    0   14   32     9   0  14    6  0.866667   166  0.333333   
10374    2    1    0    9   17     1   0   9    2  2.000000    44  0.000000   
...    ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   ...       ...   
11173    1    1    0    5    1     0   4   1    1  2.000000     1  0.000000   
11484    5    1    0   11    5    10   9   2    5  2.000000     5  0.000000   
11146    7    1    0   13    7    21   5   8    7  2.000000     7  0.000000   
12998    1    1    0    0    5     0   0   0    0  2.000000    18  0.000000   
12020    1    1    0    3    1     0   3   0    1  2.000000     1  0.000000   

       moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  
12617    1  0.800000  0.416667   0    0  48.000000       6  2.4286  
10965    0  0.000000  0.241176   0    0  65.888889      15  4.3889  
12993    0  0.000000  0.625000   0    0  22.000000       2  1.0000  
12869    0  0.886364  0.333333   2    6  26.166667      13  3.1667  
10374    0  0.000000  0.666667   0    0  21.000000       1  0.5000  
...    ...       ...       ...  ..  ...        ...     ...     ...  
11173    0  0.000000  1.000000   0    0   0.000000       1  1.0000  
11484    0  0.000000  0.480000   0    0   0.000000       1  1.0000  
11146    0  0.000000  0.333333   0    0   0.000000       1  1.0000  
12998    0  0.000000  0.000000   0    0  12.000000       0  0.0000  
12020    0  0.000000  1.000000   0    0   0.000000       1  1.0000  

[600 rows x 20 columns]

In [9]:
y_train = train['bug']
y_test = test['bug']

In [10]:
y_train

11757    0
12783    0
11552    0
12636    0
12643    3
        ..
10745    0
10621    0
11786    0
12312    1
12879    0
Name: bug, Length: 2398, dtype: int64

In [11]:
y_test

12617    0
10965    1
12993    0
12869    0
10374    1
        ..
11173    0
11484    0
11146    0
12998    0
12020    0
Name: bug, Length: 600, dtype: int64

In [12]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [13]:
x_train_np

array([[ 1.        ,  4.        ,  0.        , ...,  4.        ,
         0.        ,  0.        ],
       [ 1.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       [ 2.        ,  3.        ,  0.        , ...,  3.        ,
         1.        ,  0.5       ],
       ...,
       [ 9.        ,  2.        ,  0.        , ..., 37.77777778,
        14.        ,  2.7778    ],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 6.        ,  3.        ,  0.        , ..., 15.66666667,
         6.        ,  2.        ]])

In [14]:
x_test_np

array([[ 7.        ,  3.        ,  0.        , ..., 48.        ,
         6.        ,  2.4286    ],
       [18.        ,  1.        ,  2.        , ..., 65.88888889,
        15.        ,  4.3889    ],
       [ 2.        ,  1.        ,  0.        , ..., 22.        ,
         2.        ,  1.        ],
       ...,
       [ 7.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       [ 1.        ,  1.        ,  0.        , ..., 12.        ,
         0.        ,  0.        ],
       [ 1.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ]])

In [15]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [16]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [17]:
train_x

array([[[ 1.        ,  4.        ,  0.        , ...,  4.        ,
          0.        ,  0.        ]],

       [[ 1.        ,  1.        ,  0.        , ...,  0.        ,
          1.        ,  1.        ]],

       [[ 2.        ,  3.        ,  0.        , ...,  3.        ,
          1.        ,  0.5       ]],

       ...,

       [[ 9.        ,  2.        ,  0.        , ..., 37.77777778,
         14.        ,  2.7778    ]],

       [[ 0.        ,  1.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 6.        ,  3.        ,  0.        , ..., 15.66666667,
          6.        ,  2.        ]]])

In [18]:
test_y

array([0, 1, 0, 0, 1, 0, 0, 0, 0, 6, 1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 5,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 2, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 3, 0, 0,
       2, 1, 1, 1, 0, 0, 2, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,

In [19]:
def define_model():
    # Designing and initializing the model.
    model = Sequential()
    model.add(LSTM(200, return_sequences=True))
    model.add(LSTM(100, return_sequences=True))
    model.add(LSTM(50, dropout = 0.2, return_sequences=True))
    model.add(LSTM(20, return_sequences=False))
    model.add(Dense(1, activation = 'relu'))
    model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0005) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )
    return model

In [20]:
y = list(test_y)

In [21]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(10):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/propv121/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(train_x, train_y, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(test_x, test_y)
    total_scores.append(score) 
    predictions = model.predict(test_x)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
24/24 [==============================] - 5s 14ms/step - loss: 0.9056 - mse: 0.9056 - mae: 0.3154 - root_mean_squared_error: 0.9516 - mean_squared_logarithmic_error: 0.1332
Epoch 2/100
24/24 [==============================] - 0s 6ms/step - loss: 0.8613 - mse: 0.8613 - mae: 0.4368 - root_mean_squared_error: 0.9281 - mean_squared_logarithmic_error: 0.1361
Epoch 3/100
24/24 [==============================] - 0s 6ms/step - loss: 0.8456 - mse: 0.8456 - mae: 0.3906 - root_mean_squared_error: 0.9196 - mean_squared_logarithmic_error: 0.1249
Epoch 4/100
24/24 [==============================] - 0s 6ms/step - loss: 0.8193 - mse: 0.8193 - mae: 0.4054 - root_mean_squared_error: 0.9052 - mean_squared_logarithmic_error: 0.1257
Epoch 5/100
24/24 [==============================] - 0s 6ms/step - loss: 0.7823 - mse: 0.7823 - mae: 0.3845 - root_mean_squared_error: 0.8845 - mean_squared_logarithmic_error: 0.1182
Epoch 6/100
24/24 [==============================] - 0s 6ms/step - loss: 0

24/24 [==============================] - 0s 7ms/step - loss: 0.4744 - mse: 0.4744 - mae: 0.3029 - root_mean_squared_error: 0.6888 - mean_squared_logarithmic_error: 0.0861
Epoch 45/100
24/24 [==============================] - 0s 7ms/step - loss: 0.4551 - mse: 0.4551 - mae: 0.2949 - root_mean_squared_error: 0.6746 - mean_squared_logarithmic_error: 0.0872
Epoch 46/100
24/24 [==============================] - 0s 7ms/step - loss: 0.4846 - mse: 0.4846 - mae: 0.2862 - root_mean_squared_error: 0.6961 - mean_squared_logarithmic_error: 0.0860
Epoch 47/100
24/24 [==============================] - 0s 6ms/step - loss: 0.5108 - mse: 0.5108 - mae: 0.3026 - root_mean_squared_error: 0.7147 - mean_squared_logarithmic_error: 0.0884
Epoch 48/100
24/24 [==============================] - 0s 6ms/step - loss: 0.5602 - mse: 0.5602 - mae: 0.3218 - root_mean_squared_error: 0.7485 - mean_squared_logarithmic_error: 0.0939
Epoch 49/100
24/24 [==============================] - 0s 6ms/step - loss: 0.4842 - mse: 0.484

24/24 [==============================] - 0s 6ms/step - loss: 0.3665 - mse: 0.3665 - mae: 0.2731 - root_mean_squared_error: 0.6054 - mean_squared_logarithmic_error: 0.0727
Epoch 89/100
24/24 [==============================] - 0s 6ms/step - loss: 0.3662 - mse: 0.3662 - mae: 0.2783 - root_mean_squared_error: 0.6052 - mean_squared_logarithmic_error: 0.0723
Epoch 90/100
24/24 [==============================] - 0s 6ms/step - loss: 0.3820 - mse: 0.3820 - mae: 0.2672 - root_mean_squared_error: 0.6181 - mean_squared_logarithmic_error: 0.0720
Epoch 91/100
24/24 [==============================] - 0s 6ms/step - loss: 0.3739 - mse: 0.3739 - mae: 0.2737 - root_mean_squared_error: 0.6114 - mean_squared_logarithmic_error: 0.0746
Epoch 92/100
24/24 [==============================] - 0s 6ms/step - loss: 0.3575 - mse: 0.3575 - mae: 0.2657 - root_mean_squared_error: 0.5979 - mean_squared_logarithmic_error: 0.0715
Epoch 93/100
24/24 [==============================] - 0s 6ms/step - loss: 0.3658 - mse: 0.365

24/24 [==============================] - 0s 6ms/step - loss: 0.4726 - mse: 0.4726 - mae: 0.2947 - root_mean_squared_error: 0.6874 - mean_squared_logarithmic_error: 0.0856
Epoch 31/100
24/24 [==============================] - 0s 6ms/step - loss: 0.4592 - mse: 0.4592 - mae: 0.2947 - root_mean_squared_error: 0.6776 - mean_squared_logarithmic_error: 0.0838
Epoch 32/100
24/24 [==============================] - 0s 6ms/step - loss: 0.4459 - mse: 0.4459 - mae: 0.3004 - root_mean_squared_error: 0.6678 - mean_squared_logarithmic_error: 0.0871
Epoch 33/100
24/24 [==============================] - 0s 6ms/step - loss: 0.4472 - mse: 0.4472 - mae: 0.2990 - root_mean_squared_error: 0.6687 - mean_squared_logarithmic_error: 0.0822
Epoch 34/100
24/24 [==============================] - 0s 6ms/step - loss: 0.4477 - mse: 0.4477 - mae: 0.2942 - root_mean_squared_error: 0.6691 - mean_squared_logarithmic_error: 0.0841
Epoch 35/100
24/24 [==============================] - 0s 6ms/step - loss: 0.4485 - mse: 0.448

24/24 [==============================] - 0s 6ms/step - loss: 0.3584 - mse: 0.3584 - mae: 0.2732 - root_mean_squared_error: 0.5987 - mean_squared_logarithmic_error: 0.0733
Epoch 74/100
24/24 [==============================] - 0s 6ms/step - loss: 0.4045 - mse: 0.4045 - mae: 0.2929 - root_mean_squared_error: 0.6360 - mean_squared_logarithmic_error: 0.0815
Epoch 75/100
24/24 [==============================] - 0s 6ms/step - loss: 0.4240 - mse: 0.4240 - mae: 0.2729 - root_mean_squared_error: 0.6512 - mean_squared_logarithmic_error: 0.0761
Epoch 76/100
24/24 [==============================] - 0s 6ms/step - loss: 0.3877 - mse: 0.3877 - mae: 0.2973 - root_mean_squared_error: 0.6227 - mean_squared_logarithmic_error: 0.0797
Epoch 77/100
24/24 [==============================] - 0s 6ms/step - loss: 0.3885 - mse: 0.3885 - mae: 0.2873 - root_mean_squared_error: 0.6233 - mean_squared_logarithmic_error: 0.0783
Epoch 78/100
24/24 [==============================] - 0s 6ms/step - loss: 0.3870 - mse: 0.387

...

24/24 [==============================] - 0s 6ms/step - loss: 0.5347 - mse: 0.5347 - mae: 0.2977 - root_mean_squared_error: 0.7312 - mean_squared_logarithmic_error: 0.0951
Epoch 20/100
24/24 [==============================] - 0s 6ms/step - loss: 0.5222 - mse: 0.5222 - mae: 0.2915 - root_mean_squared_error: 0.7226 - mean_squared_logarithmic_error: 0.0896
Epoch 21/100
24/24 [==============================] - 0s 6ms/step - loss: 0.5599 - mse: 0.5599 - mae: 0.2795 - root_mean_squared_error: 0.7483 - mean_squared_logarithmic_error: 0.0913
Epoch 22/100
24/24 [==============================] - 0s 6ms/step - loss: 0.5814 - mse: 0.5814 - mae: 0.2936 - root_mean_squared_error: 0.7625 - mean_squared_logarithmic_error: 0.0950
Epoch 23/100
24/24 [==============================] - 0s 6ms/step - loss: 0.5504 - mse: 0.5504 - mae: 0.3092 - root_mean_squared_error: 0.7419 - mean_squared_logarithmic_error: 0.0963
Epoch 24/100
24/24 [==============================] - 0s 6ms/step - loss: 0.5344 - mse: 0.534

24/24 [==============================] - 0s 6ms/step - loss: 0.4171 - mse: 0.4171 - mae: 0.2814 - root_mean_squared_error: 0.6458 - mean_squared_logarithmic_error: 0.0778
Epoch 64/100
24/24 [==============================] - 0s 6ms/step - loss: 0.4007 - mse: 0.4007 - mae: 0.2878 - root_mean_squared_error: 0.6330 - mean_squared_logarithmic_error: 0.0792
Epoch 65/100
24/24 [==============================] - 0s 6ms/step - loss: 0.3997 - mse: 0.3997 - mae: 0.2812 - root_mean_squared_error: 0.6322 - mean_squared_logarithmic_error: 0.0753
Epoch 66/100
24/24 [==============================] - 0s 6ms/step - loss: 0.3911 - mse: 0.3911 - mae: 0.2831 - root_mean_squared_error: 0.6254 - mean_squared_logarithmic_error: 0.0757
Epoch 67/100
24/24 [==============================] - 0s 6ms/step - loss: 0.3711 - mse: 0.3711 - mae: 0.2776 - root_mean_squared_error: 0.6092 - mean_squared_logarithmic_error: 0.0740
Epoch 68/100
24/24 [==============================] - 0s 6ms/step - loss: 0.3883 - mse: 0.388

24/24 [==============================] - 0s 6ms/step - loss: 0.7730 - mse: 0.7730 - mae: 0.3871 - root_mean_squared_error: 0.8792 - mean_squared_logarithmic_error: 0.1186
Epoch 6/100
24/24 [==============================] - 0s 6ms/step - loss: 0.7429 - mse: 0.7429 - mae: 0.3686 - root_mean_squared_error: 0.8619 - mean_squared_logarithmic_error: 0.1158
Epoch 7/100
24/24 [==============================] - 0s 6ms/step - loss: 0.6935 - mse: 0.6935 - mae: 0.3568 - root_mean_squared_error: 0.8327 - mean_squared_logarithmic_error: 0.1122
Epoch 8/100
24/24 [==============================] - 0s 6ms/step - loss: 0.6840 - mse: 0.6840 - mae: 0.3382 - root_mean_squared_error: 0.8270 - mean_squared_logarithmic_error: 0.1113
Epoch 9/100
24/24 [==============================] - 0s 6ms/step - loss: 0.6280 - mse: 0.6280 - mae: 0.3235 - root_mean_squared_error: 0.7925 - mean_squared_logarithmic_error: 0.1030
Epoch 10/100
24/24 [==============================] - 0s 6ms/step - loss: 0.5933 - mse: 0.5933 - 

24/24 [==============================] - 0s 6ms/step - loss: 0.4236 - mse: 0.4236 - mae: 0.2781 - root_mean_squared_error: 0.6509 - mean_squared_logarithmic_error: 0.0808
Epoch 50/100
24/24 [==============================] - 0s 6ms/step - loss: 0.4339 - mse: 0.4339 - mae: 0.2707 - root_mean_squared_error: 0.6587 - mean_squared_logarithmic_error: 0.0803
Epoch 51/100
24/24 [==============================] - 0s 6ms/step - loss: 0.4442 - mse: 0.4442 - mae: 0.2864 - root_mean_squared_error: 0.6665 - mean_squared_logarithmic_error: 0.0816
Epoch 52/100
24/24 [==============================] - 0s 6ms/step - loss: 0.4476 - mse: 0.4476 - mae: 0.2877 - root_mean_squared_error: 0.6690 - mean_squared_logarithmic_error: 0.0828
Epoch 53/100
24/24 [==============================] - 0s 6ms/step - loss: 0.4424 - mse: 0.4424 - mae: 0.2816 - root_mean_squared_error: 0.6651 - mean_squared_logarithmic_error: 0.0831
Epoch 54/100
24/24 [==============================] - 0s 6ms/step - loss: 0.4581 - mse: 0.458

24/24 [==============================] - 0s 6ms/step - loss: 0.3086 - mse: 0.3086 - mae: 0.2627 - root_mean_squared_error: 0.5555 - mean_squared_logarithmic_error: 0.0686
Epoch 94/100
24/24 [==============================] - 0s 6ms/step - loss: 0.2971 - mse: 0.2971 - mae: 0.2591 - root_mean_squared_error: 0.5451 - mean_squared_logarithmic_error: 0.0681
Epoch 95/100
24/24 [==============================] - 0s 6ms/step - loss: 0.3152 - mse: 0.3152 - mae: 0.2537 - root_mean_squared_error: 0.5614 - mean_squared_logarithmic_error: 0.0674
Epoch 96/100
24/24 [==============================] - 0s 6ms/step - loss: 0.3248 - mse: 0.3248 - mae: 0.2683 - root_mean_squared_error: 0.5700 - mean_squared_logarithmic_error: 0.0707
Epoch 97/100
24/24 [==============================] - 0s 6ms/step - loss: 0.3085 - mse: 0.3085 - mae: 0.2616 - root_mean_squared_error: 0.5554 - mean_squared_logarithmic_error: 0.0695
Epoch 98/100
24/24 [==============================] - 0s 6ms/step - loss: 0.3452 - mse: 0.345

In [22]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/propv121/'

In [23]:
total_training_time

[22.641123056411743,
 21.008378505706787,
 21.224088191986084,
 21.105833053588867,
 21.279627323150635,
 22.045073747634888,
 22.229191541671753,
 21.65488576889038,
 21.372720956802368,
 21.764097690582275]

In [24]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 7.210833994547526


In [25]:
total_training_loss_history

[{'loss': [0.9055903553962708,
   0.8613436222076416,
   0.8455747365951538,
   0.8193340301513672,
   0.7823358774185181,
   0.7322222590446472,
   0.6900461316108704,
   0.6553346514701843,
   0.6490308046340942,
   0.6131928563117981,
   0.5921089053153992,
   0.584465742111206,
   0.5648611187934875,
   0.564647376537323,
   0.5359911918640137,
   0.5491984486579895,
   0.5232706069946289,
   0.5194351673126221,
   0.5106741786003113,
   0.5056743025779724,
   0.5202834010124207,
   0.5014334917068481,
   0.4975391924381256,
   0.5228205323219299,
   0.49347880482673645,
   0.47129637002944946,
   0.4797327220439911,
   0.5116262435913086,
   0.47001418471336365,
   0.43920984864234924,
   0.4577813744544983,
   0.4597145617008209,
   0.47458508610725403,
   0.45346304774284363,
   0.448659747838974,
   0.4221579134464264,
   0.45473065972328186,
   0.47455164790153503,
   0.4626057744026184,
   0.48460304737091064,
   0.4743559658527374,
   0.4895952045917511,
   0.457851231098175

In [26]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(10):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 0.1410494198401769
avg mae of training: 0.0907772699991862


In [27]:
total_scores

[[0.6306520104408264,
  0.6306520104408264,
  0.3180374801158905,
  0.7941359877586365,
  0.12403959780931473],
 [0.5935705304145813,
  0.5935705304145813,
  0.33590665459632874,
  0.7704352736473083,
  0.12526927888393402],
 [0.6115629076957703,
  0.6115629076957703,
  0.341449111700058,
  0.7820248603820801,
  0.12266577035188675],
 [0.6519631743431091,
  0.6519631743431091,
  0.3557155132293701,
  0.8074423670768738,
  0.13042545318603516],
 [0.581951916217804,
  0.581951916217804,
  0.3014181852340698,
  0.7628577351570129,
  0.11559635400772095],
 [0.6681485176086426,
  0.6681485176086426,
  0.3547055721282959,
  0.8174034953117371,
  0.13295473158359528],
 [0.47999998927116394,
  0.47999998927116394,
  0.20333333313465118,
  0.6928203105926514,
  0.1143576055765152],
 [0.6537876129150391,
  0.6537876129150391,
  0.32738247513771057,
  0.8085713386535645,
  0.12569066882133484],
 [0.4933643639087677,
  0.4933643639087677,
  0.2898865044116974,
  0.7023990154266357,
  0.10732918977

In [28]:
a = [[1.1072313785552979,
  1.1072313785552979,
  0.45697730779647827,
  1.052250623703003,
  0.1505131721496582],
 [1.0949896574020386,
  1.0949896574020386,
  0.4038865566253662,
  1.0464175939559937,
  0.12835438549518585],
 [1.1006139516830444,
  1.1006139516830444,
  0.4063127636909485,
  1.04910147190094,
  0.13611803948879242],
 [1.0576918125152588,
  1.0576918125152588,
  0.40475377440452576,
  1.0284414291381836,
  0.132333904504776],
 [1.087944507598877,
  1.087944507598877,
  0.45055320858955383,
  1.0430457592010498,
  0.14320626854896545],
 [1.1228747367858887,
  1.1228747367858887,
  0.4579310417175293,
  1.0596578121185303,
  0.15697523951530457],
 [1.1081269979476929,
  1.1081269979476929,
  0.41065943241119385,
  1.0526760816574097,
  0.13614483177661896],
 [1.0938034057617188,
  1.0938034057617188,
  0.38377878069877625,
  1.0458505153656006,
  0.12563826143741608],
 [1.046379804611206,
  1.046379804611206,
  0.3927305042743683,
  1.0229270458221436,
  0.12750454246997833],
 [1.077500343322754,
  1.077500343322754,
  0.4189930856227875,
  1.0380271673202515,
  0.1433822512626648],
 [1.0831170082092285,
  1.0831170082092285,
  0.39570093154907227,
  1.04072904586792,
  0.12718339264392853],
 [1.0793119668960571,
  1.0793119668960571,
  0.4216558337211609,
  1.0388994216918945,
  0.13091830909252167],
 [1.0471224784851074,
  1.0471224784851074,
  0.41589176654815674,
  1.0232900381088257,
  0.1298009604215622],
 [1.112039566040039,
  1.112039566040039,
  0.432132750749588,
  1.0545328855514526,
  0.14624285697937012],
 [1.117440104484558,
  1.117440104484558,
  0.3917667865753174,
  1.057090401649475,
  0.12741179764270782],
 [1.0524473190307617,
  1.0524473190307617,
  0.4215909242630005,
  1.0258885622024536,
  0.1336907595396042],
 [1.0733959674835205,
  1.0733959674835205,
  0.42276591062545776,
  1.0360482931137085,
  0.13468603789806366],
 [1.1121301651000977,
  1.1121301651000977,
  0.4868672788143158,
  1.054575800895691,
  0.16745002567768097],
 [1.0329020023345947,
  1.0329020023345947,
  0.3979037404060364,
  1.0163178443908691,
  0.12879155576229095],
 [1.0707591772079468,
  1.0707591772079468,
  0.4306138753890991,
  1.034774899482727,
  0.13812918961048126],
 [1.075225830078125,
  1.075225830078125,
  0.40528926253318787,
  1.0369309186935425,
  0.12670773267745972],
 [1.083695411682129,
  1.083695411682129,
  0.4407236576080322,
  1.0410069227218628,
  0.13586269319057465],
 [1.0840530395507812,
  1.0840530395507812,
  0.4021536111831665,
  1.0411787033081055,
  0.12666361033916473],
 [1.1058012247085571,
  1.1058012247085571,
  0.3986241817474365,
  1.0515708923339844,
  0.12572358548641205],
 [1.1244145631790161,
  1.1244145631790161,
  0.4341464340686798,
  1.0603841543197632,
  0.14787453413009644],
 [1.0968248844146729,
  1.0968248844146729,
  0.41786739230155945,
  1.047294020652771,
  0.13397936522960663],
 [1.0843594074249268,
  1.0843594074249268,
  0.4130922853946686,
  1.0413258075714111,
  0.13689492642879486],
 [1.0843403339385986,
  1.0843403339385986,
  0.4019320011138916,
  1.0413166284561157,
  0.13194996118545532],
 [1.0823019742965698,
  1.0823019742965698,
  0.3958171606063843,
  1.0403374433517456,
  0.13313862681388855],
 [1.056094765663147,
  1.056094765663147,
  0.4249243140220642,
  1.0276646614074707,
  0.13509279489517212]]

In [29]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])

In [30]:
test_mae

[0.45697730779647827,
 0.4038865566253662,
 0.4063127636909485,
 0.40475377440452576,
 0.45055320858955383,
 0.4579310417175293,
 0.41065943241119385,
 0.38377878069877625,
 0.3927305042743683,
 0.4189930856227875]

In [31]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(10):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 0.19763911962509156
avg mae: 0.10487041423718134
avg are: 0.04196296234925588


In [32]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.3758906275033951
median testing mse: 0.6025667190551758


In [33]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.2706577330827713
median testing mae: 0.32283003628253937


In [34]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.12618055939674377


In [35]:
test_ARE

[0.12430555,
 0.11972222,
 0.12625,
 0.12777779,
 0.12638889,
 0.12555555,
 0.12833333,
 0.12944444,
 0.12611112,
 0.125]